# Trabalho Final Congnite Enviromets

- Nome: Felipe Fabossi | RM: 353427
- Nome: Paola Lemes | RM: 336523
- Nome: Yago Angelini | RM: 354173

## Instalações e Importações

In [ ]:
!pip install boto3

In [ ]:
!pip install Pillow

In [ ]:
!pip install openai requests pillow

In [36]:
import boto3
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageDraw, ImageFont
import openai
import requests
import base64
import io
from google.colab import files
import matplotlib.pyplot as plt
import os
import unicodedata
import json

## Extração de texto

### Extração de NOME e CPF na CNH

In [ ]:
# Chaves de acesso AWS
print("Digite suas chaves de acesso AWS:")
ACCESS_ID = input("Digite seu ACCESS_ID: ")
ACCESS_KEY = input("Digite seu ACCESS_KEY: ")
region = "us-east-1"

Função para extrair o texto da imagem da CNH utilizando o serviço em cloud da AWS e retornando o texto contido na imagem

In [38]:
# Função para processar o arquivo e retornar o texto extraído
def processar_arquivo(file_name):
    # Lendo o arquivo em imagem para exibir
    img = mpimg.imread(file_name)
    imgplot = plt.imshow(img)

    plt.axis('off')
    plt.show()

    with open(file_name, "rb") as file:
        img_test = file.read()
        bytes_test = bytearray(img_test)

    # Conectando ao AWS Textract
    session = boto3.Session(aws_access_key_id=ACCESS_ID, aws_secret_access_key=ACCESS_KEY)
    client = session.client('textract', region_name=region)
    response = client.analyze_document(Document={'Bytes': bytes_test}, FeatureTypes=['FORMS'])

    blocks = response["Blocks"]

    texto_extraido = []  # Lista para armazenar os textos extraídos

    for block in blocks:
        if block["BlockType"] == "WORD" and int(block["Confidence"]) > 50:
            texto_extraido.append(block["Text"])

    return " ".join(texto_extraido)  # Retorna o texto extraído como uma única string

In [ ]:
print("Faça upload da imagem da CNH (PNG, JPEG, etc.):")
uploaded = files.upload()
foto_cnh = next(iter(uploaded))

texto_cnh = processar_arquivo(foto_cnh)
print(texto_cnh)

Função para extrair o NOME e CPF do texto obtido através da imagem da CNH

In [40]:
import re

def extrair_nome_cpf(texto_extraido):
    # Expressão regular para encontrar o nome (após "HABILITAÇÃO" e antes de uma data)
    padrao_nome = r"HABILITAÇÃO\s+([A-Z\s]+?)\s+\d{2}/\d{2}/\d{4}"
    nome = re.search(padrao_nome, texto_extraido)
    nome = nome.group(1).strip() if nome else "Nome não encontrado"

    # Expressão regular para encontrar o CPF (após "HAB" e formado por XXX.XXX.XXX-XX)
    padrao_cpf = r"HAB\s+(\d{3}\.\d{3}\.\d{3}-\d{2})"
    cpf = re.search(padrao_cpf, texto_extraido)
    cpf = cpf.group(1) if cpf else "CPF não encontrado"

    return {"nome": nome, "cpf": cpf}


In [ ]:
dados_cnh = extrair_nome_cpf(texto_cnh)
print(dados_cnh)

nome_cnh = dados_cnh["nome"]
cpf_cnh = dados_cnh["cpf"]

### Extração de NOME e ENDEREÇO em comprovante/conta

In [42]:
# Função para extrair texto com GPT-4o
def extrair_texto_com_openai(imagem_path, api_key):
    # Configurar a API da OpenAI
    openai.api_key = api_key

    # Carregar imagem
    img = Image.open(imagem_path)

    # Detectar o formato da imagem
    formato = os.path.splitext(imagem_path)[1].lower().replace('.', '')
    if formato not in ['jpeg', 'jpg', 'png']:
        formato = 'png'

    # Converter imagem para base64
    buffer = io.BytesIO()
    img.save(buffer, format=formato.upper())
    imagem_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

    try:
        # Extrair o texto da imagem
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "Por favor, descreva detalhadamente o que você vê nesta imagem, incluindo qualquer texto visível."
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/{formato};base64,{imagem_base64}"
                            }
                        }
                    ]
                }
            ],
            max_tokens=1000
        )
        texto_extraido = response.choices[0].message.content

        # Extrair informações estruturadas
        segunda_resposta = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": "Você é um assistente que analisa documentos."
                },
                {
                    "role": "user",
                    "content": f"Com base no texto a seguir, extraia o nome completo da pessoa e o endereço completo em formato JSON com campos 'nome' e 'endereco':\n\n{texto_extraido}"
                }
            ]
        )
        resultado = segunda_resposta.choices[0].message.content

        # Extrair o JSON
        match = re.search(r'\{.*\}', resultado, re.DOTALL)
        if match:
            json_str = match.group(0)
            dados = json.loads(json_str)
            return dados.get('nome', ''), dados.get('endereco', '')
        else:
            return None, None

    except Exception as e:
        print(f"Erro ao processar a imagem: {e}")
        return None, None




### Comparar nomes

In [43]:
def comparar_nomes(nome1, nome2):
    if not nome1 or not nome2:
        return False

    def normalizar_nome(nome):
        nome = nome.upper()
        nome = unicodedata.normalize('NFKD', nome).encode('ASCII', 'ignore').decode('ASCII')
        nome = re.sub(r'[^\w\s]', '', nome)
        return ' '.join(nome.split())

    nome1_norm = normalizar_nome(nome1)
    nome2_norm = normalizar_nome(nome2)

    if nome1_norm == nome2_norm:
        return True
    if nome1_norm in nome2_norm or nome2_norm in nome1_norm:
        return True

    palavras1 = set(nome1_norm.split())
    palavras2 = set(nome2_norm.split())
    palavras_comuns = palavras1.intersection(palavras2)
    min_palavras = min(len(palavras1), len(palavras2))

    if min_palavras > 0 and len(palavras_comuns) / min_palavras >= 0.7:
        return True

    return False


### Validação e comparação dos dados extraídos da conta e da CNH

In [ ]:
# Função principal
def validar_conta(nome_cnh):
    print("Carregue a imagem da conta/fatura:")
    uploaded = files.upload()
    imagem_path = next(iter(uploaded))

    api_key = input("Digite sua chave de API da OpenAI: ")

    nome_conta, endereco_conta = extrair_texto_com_openai(imagem_path, api_key)

    print("\n=== RESULTADOS ===")
    print(f"Nome da CNH: {nome_cnh}")
    print(f"Nome da conta: {nome_conta or 'Não detectado'}")
    print(f"Endereço: {endereco_conta or 'Não detectado'}")

    if nome_conta:
        correspondencia = comparar_nomes(nome_cnh, nome_conta)
        print(f"Nomes correspondem: {'SIM' if correspondencia else 'NÃO'}")
        return nome_conta, endereco_conta, correspondencia
    else:
        print("Não foi possível realizar a comparação.")
        return None, endereco_conta, False

try:
    nome_conta, endereco_conta, correspondencia = validar_conta(nome_cnh)
except NameError:
    nome_cnh = input("Digite o nome extraído da CNH: ")
    nome_conta, endereco_conta, correspondencia = validar_conta(nome_cnh)

## Comparação de Faces

### Extração da face

In [ ]:
print("Faça upload de uma foto sua: ")
uploaded = files.upload()
foto_selfie = next(iter(uploaded))


In [46]:
# convertendo a imagem de origem no formato binário
with open(foto_cnh, "rb") as file:
  img_file = file.read()
  bytes_file_source = bytearray(img_file)

# convertendo a imagem alvo no formato binário
with open(foto_selfie, "rb") as file:
  img_file = file.read()
  bytes_file_target = bytearray(img_file)

# abrindo a sessão
session = boto3.Session(aws_access_key_id=ACCESS_ID, aws_secret_access_key= ACCESS_KEY)

# criando o cliente
client = session.client("rekognition", region_name=region)

# realizando a requisição
response = client.compare_faces(
    SourceImage={'Bytes': bytes_file_source},
    TargetImage={'Bytes': bytes_file_target},
)

In [ ]:
# imagem de origem
image = Image.open(foto_cnh)
image.thumbnail((500, 500))

# obtendo tamanho para cálculo de posicionamento
imgWidth, imgHeight = image.size

# obtendo imagem para anotação
draw = ImageDraw.Draw(image)

# obtendo a caixa delimitadora
box = response["SourceImageFace"]["BoundingBox"]

top = imgHeight * box['Top']
left = imgWidth * box['Left']
width = imgWidth * box['Width']
height = imgHeight * box['Height']

print("Left: " + "{0:.0f}".format(left))
print("Top: " + "{0:.0f}".format(top))
print("Face Width: " + "{0:.0f}".format(width))
print("Face Height: " + "{0:.0f}".format(height))

draw.rectangle([left,top, left + width, top + height], outline='#00d400')

display(image)

### Comparação

In [ ]:
# imagem alvo
image = Image.open(foto_selfie)
image.thumbnail((500, 500))
imgWidth, imgHeight = image.size

# obtendo imagem para anotação
draw = ImageDraw.Draw(image)

# analisando cada detecção de face
for item_match in response["FaceMatches"]:
  print("Face Localizada: " + str(item_match))

  # caixa delimitadora
  box = item_match["Face"]["BoundingBox"]

  top = imgHeight * box['Top']
  left = imgWidth * box['Left']
  width = imgWidth * box['Width']
  height = imgHeight * box['Height']

  print("Left: " + "{0:.0f}".format(left))
  print("Top: " + "{0:.0f}".format(top))
  print("Face Width: " + "{0:.0f}".format(width))
  print("Face Height: " + "{0:.0f}".format(height))

  draw.rectangle([left,top, left + width, top + height], outline='#00d400', )
  draw.text((left, top), "Similaridade: "+ str(item_match["Similarity"]))

  print("Similaridade: " + str(item_match["Similarity"]))
  display(image)